In [ ]:
host= '' 
port=1000 
username=''

In [1]:
# ------------------------------------------ FINAL ----------------------------------
from pyhive import presto
import pandas as pd
import openpyxl

cursor = presto.connect(host= host, port=port, username=username).cursor() 
df = pd.DataFrame()

# with open('adcs_0.sql', 'r') as f:
#     sql = f.read()

msisdn =  '8801XXXXXXXX\',\'8801YYYYYYYY'

date = '2019-05-28'

filename= "Final.xlsx"

data_details =('''Select cost/10 price, pack_id, volume/1024 MB,phone_number,partition_date from data_details 
        where phone_number in (\''''+msisdn+'''\')
        and partition_date=date\''''+date+'''\'
        order by timestamp desc''')

bundle_details = ('''Select bundle_id, charge/10 price, disbursement_id1, disbursement1_volume,phone_number,partition_date from bundle_details
        where phone_number in (\''''+msisdn+'''\')
        and partition_date=date\''''+date+'''\'
        order by timestamp desc''')

# x=cursor.fetchall()
# cursor.fetchall()
# df=pd.DataFrame(x)


cursor.execute(data_details)
qFetch = cursor.fetchall()
df_data_details=pd.DataFrame.from_records(qFetch, columns = [i[0] for i in cursor.description])
df_data_details['phone_number'] = pd.to_numeric(df_data_details['phone_number'])


cursor.execute(bundle_details)
qFetch = cursor.fetchall()
df_bundle_details=pd.DataFrame.from_records(qFetch, columns = [i[0] for i in cursor.description])
df_bundle_details['phone_number'] = pd.to_numeric(df_bundle_details['phone_number'])



with pd.ExcelWriter(filename) as writer:# doctest: +SKIP
    df_data_details.to_excel(writer, sheet_name='df_data_details', index=False)
    df_bundle_details.to_excel(writer, sheet_name='df_bundle_details', index=False)


import os 

# open the file
os.startfile('Final.xlsx')

In [ ]:
# Import necessary libraries from openpyxl
from openpyxl import load_workbook
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import PatternFill

filename='Final.xlsx'
# Load the existing Excel workbook
workbook = load_workbook(filename=filename, data_only=True)

# Open the workbook and select the sheet named "Sheet1"
sheet = workbook["df_data_details"]

# Copy the value from cell D2 to cell D3
sheet["G2"].value = sheet["E2"].value

# Perform calculations in various cells using formulas
sheet["I2"].value = "=C3-C2"

# Set titles for the columns
sheet["C1"].value = "DATA VOL in MB"

# Apply color formatting to certain cells
blue_fill = PatternFill(start_color="0000FF", end_color="0000FF", fill_type="solid")
yellow_fill = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")

sheet["I2"].fill = yellow_fill 
sheet["D1"].fill = blue_fill

# Save the modified workbook
workbook.save(filename=filename)

# Open the modified Excel file
import os
os.startfile(filename)
